In [1]:
import re
import pandas as pd
import numpy as np
import itertools

In [2]:
def check_base_info(input_path, filename, output_path, minimal=False):
    df = pd.read_csv(input_path + filename + '.csv',  engine = 'python')
    col_names = df.columns.to_series()
    col_dtypes = df.dtypes
    col_na_count = df.isnull().sum()
    col_describe = df.describe().T
    col_unique = df.nunique()

    base_info = pd.concat([col_names, col_dtypes, col_na_count, col_unique], 
                          axis=1, keys=['col_names', 'col_dtypes', 
                                        'col_na_count', 'col_unique'], sort=False)
    base_info = pd.concat([base_info, col_describe], axis=1, sort=False)

    base_info.to_csv(output_path + filename + 'base_info.csv')
    
    df_report = ProfileReport(df, minimal=minimal)
    df_report.to_file(output_file = output_path + filename + '.html')

def check_missing(df):
    col_names = df.columns.to_series()
    col_dtypes = df.dtypes
    col_na_count = df.isnull().sum()
    col_describe = df.describe().T
    col_unique = df.nunique()

    base_info = pd.concat([col_names, col_dtypes, col_na_count, col_unique], 
                          axis=1, keys=['col_names', 'col_dtypes', 
                                        'col_na_count', 'col_unique'], sort=False)
    base_info = pd.concat([base_info, col_describe], axis=1, sort=False)
    
    return base_info

# 创建比赛ID，所有队伍交叉可能（不考虑区域），男队ID 为1000-1999, 数据中 ID数为1101-1467

In [3]:
input_path_mdata = f'./input/rawdata/MDataFiles_Stage1/'
input_path_mevents = f'./input/rawdata/'
output_path = f'./output/'

In [4]:
list_TeamID = [x for x in range(1101, 1468)]
list_Season = [x for x in range(1985, 2020)]

ID = []
Season = []
TeamID_a = []
TeamID_b = []

for item in  itertools.product(list_Season, list_TeamID, list_TeamID):
    if item[1] < item[2]:
        _ = str(item[0]) + '_' + str(item[1]) + '_' + str(item[2]) 
        ID.append(_)
        Season.append(item[0])
        TeamID_a.append(item[1])
        TeamID_b.append(item[2])

        
df_datamart = pd.concat([pd.Series(ID), pd.Series(Season), 
                pd.Series(TeamID_a), pd.Series(TeamID_b)], 
               axis=1, keys=['ID', 'Season', 'TeamID_a', 'TeamID_b' ])

In [5]:
check_missing(df_datamart)

,col_names,col_dtypes,col_na_count,col_unique,count,mean,std,min,25%,50%,75%,max
ID,ID,object,0,2350635,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Season,Season,int64,0,35,2350635.0,2002.000000,10.099507,1985.0,1993.0,2002.0,2011.0,2019.0
TeamID_a,TeamID_a,int64,0,366,2350635.0,1222.666667,86.384173,1101.0,1150.0,1208.0,1284.0,1466.0
TeamID_b,TeamID_b,int64,0,366,2350635.0,1345.333333,86.384173,1102.0,1284.0,1360.0,1418.0,1467.0


In [6]:
print(df_datamart.head(100))

print(df_datamart.shape)

                ID  Season  TeamID_a  TeamID_b
0   1985_1101_1102    1985      1101      1102
1   1985_1101_1103    1985      1101      1103
2   1985_1101_1104    1985      1101      1104
3   1985_1101_1105    1985      1101      1105
4   1985_1101_1106    1985      1101      1106
..             ...     ...       ...       ...
95  1985_1101_1197    1985      1101      1197
96  1985_1101_1198    1985      1101      1198
97  1985_1101_1199    1985      1101      1199
98  1985_1101_1200    1985      1101      1200
99  1985_1101_1201    1985      1101      1201

[100 rows x 4 columns]
(2350635, 4)


# 加入target_p1，target_p2 此部分为对阵比赛结果，target_p1为分差，用score_diff表示，target_p2为胜负，用win表示
### [MRegularSeasonDetaileResults.csv] [MNCAATourneyDetailedResults.csv]两个文件中提取出胜负结果及分差，另加一列来区别常规赛和淘汰赛

In [7]:
df_regular_results = pd.read_csv(input_path_mdata + 'MRegularSeasonDetailedResults.csv', engine = 'python')
df_tourney_results = pd.read_csv(input_path_mdata + 'MNCAATourneyDetailedResults.csv', engine = 'python')

In [8]:
df_regular_results.head(5)

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,10,1104,68,1328,62,N,0,27,58,...,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,N,0,26,62,...,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,N,0,24,58,...,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,N,0,18,38,...,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,N,0,30,61,...,16,17,27,21,15,12,10,7,1,14


In [9]:
df_tourney_results.head(5)

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,134,1421,92,1411,84,N,1,32,69,...,31,14,31,17,28,16,15,5,0,22
1,2003,136,1112,80,1436,51,N,0,31,66,...,16,7,7,8,26,12,17,10,3,15
2,2003,136,1113,84,1272,71,N,0,31,59,...,28,14,21,20,22,11,12,2,5,18
3,2003,136,1141,79,1166,73,N,0,29,53,...,17,12,17,14,17,20,21,6,6,21
4,2003,136,1143,76,1301,74,N,1,27,64,...,21,15,20,10,26,16,14,5,8,19


### 说明：文件 [MRegularSeasonDetaileResults.csv] [MNCAATourneyDetailedResults.csv] 是文件 【MRegularSeasonCompactResults.csv】【MNCAATourneyCompactResults.csv】的扩展版本
### 前8列完全相同，后面追加的为整场比赛中的统计数据（此处要跟event区分开，event是发生每个活动都记录，每场比赛统计多次， 而detaile是每场比赛结束时，统计一次总数）
### WTO-turnovers committed- 赢队的失误次数 WBlk-blocks- 赢队的盖帽次数
### 每列说明： WFGM - 投篮投中次数（所有，包括2分，3分，不含罚篮） WFGA-投篮次数（比如5投3中 的5） WFGM3-3分球投中次数 WFGA3-3分球投篮次数 WFTM-罚球中次数 WFTA-罚球次数 
### WOR-进攻篮板 WDR-防守篮板 WAst-助攻 WTO-turnovers committed？？ WStl-抢断 WBlk-blocks？？ WPF-犯规

In [10]:
#Add a lable variable to the two datasets

df_regular_results['Label'] = 'Reg'
df_tourney_results['Label']= 'Tourney'

In [11]:
df_regular_results.shape

(87504, 35)

## Feature Engineering - Adding features for DataMart

### 1. Adding Score Difference

In [12]:
def score_diff(dfin):
    df = dfin.copy()
    df['id_diff'] = df['WTeamID'] - df['LTeamID'] #Variable used to create ID matched with datamart
    '''Rule: if Win team has smaller ID number than Lost team - > Score_diff = Wscore - LScore
            else: Score_diff = LScore - Wscore since the team ID order will be exchanged'''
    '''[Wloc] identifies the "location" of the winning team. 
            If the winning team was the home team, this value will be "H". 
            If the winning team was the visiting team, this value will be "A". 
            If it was played on a neutral court, then this value will be "N".'''

    df['win'] = df.apply(lambda x: 1 if x.id_diff<0 else 0, axis=1) 
    df['LLoc'] = df.apply(lambda x: 'H' if x.WLoc == 'A' else('A' if x.WLoc == 'H' else 'N'), axis =1)
    
    df['Team_a_Loc'] = df.apply(lambda x: x.WLoc if x.id_diff<0 else x.LLoc, axis=1)
    df['Team_a_FGM2_rate'] = df.apply(lambda x: (x.WFGM - x.WFGM3) / (x.WFGA - x.WFGA3) if x.id_diff <0 else (x.LFGM - x.LFGM3) / (x.LFGA - x.LFGA3), axis=1) 
    df['Team_a_FGM3_rate'] = df.apply(lambda x: x.WFGM3 / x.WFGA3 if x.id_diff <0 else x.LFGM3 / x.LFGA3, axis=1)
    df['Team_a_FTM_count'] = df.apply(lambda x: x.WFTA if x.id_diff <0 else x.LFTA, axis=1) 
    df['Team_a_OR'] = df.apply(lambda x: x.WOR if x.id_diff<0 else x.LOR, axis =1)
    df['Team_a_DR'] = df.apply(lambda x: x.WDR if x.id_diff<0 else x.LDR, axis =1)
    df['Team_a_Ast'] = df.apply(lambda x: x.WAst if x.id_diff<0 else x.LAst, axis =1)
    df['Team_a_TO'] = df.apply(lambda x: x.WTO if x.id_diff<0 else x.LTO, axis =1)
    df['Team_a_Stl'] = df.apply(lambda x: x.WStl if x.id_diff<0 else x.LStl, axis =1)
    df['Team_a_Blk'] = df.apply(lambda x: x.WBlk if x.id_diff<0 else x.LBlk, axis =1)
    df['Team_a_PF'] = df.apply(lambda x: x.WPF if x.id_diff<0 else x.LPF, axis =1)
    
    df['Team_b_Loc'] = df.apply(lambda x: x.LLoc if x.id_diff<0 else x.WLoc, axis=1)
    df['Team_b_FGM2_rate'] = df.apply(lambda x: (x.LFGM - x.LFGM3) / (x.LFGA - x.LFGA3) if x.id_diff <0 else (x.WFGM - x.WFGM3) / (x.WFGA - x.WFGA3), axis=1) 
    df['Team_b_FGM3_rate'] = df.apply(lambda x: x.LFGM3 / x.LFGA3 if x.id_diff <0 else x.WFGM3 / x.WFGA3, axis=1)
    df['Team_b_FTM_count'] = df.apply(lambda x: x.LFTA if x.id_diff <0 else x.WFTA, axis=1) 
    df['Team_b_OR'] = df.apply(lambda x: x.LOR if x.id_diff<0 else x.WOR, axis =1)
    df['Team_b_DR'] = df.apply(lambda x: x.LDR if x.id_diff<0 else x.WDR, axis =1)
    df['Team_b_Ast'] = df.apply(lambda x: x.LAst if x.id_diff<0 else x.WAst, axis =1)
    df['Team_b_TO'] = df.apply(lambda x: x.LTO if x.id_diff<0 else x.WTO, axis =1)
    df['Team_b_Stl'] = df.apply(lambda x: x.LStl if x.id_diff<0 else x.WStl, axis =1)
    df['Team_b_Blk'] = df.apply(lambda x: x.LBlk if x.id_diff<0 else x.WBlk, axis =1)
    df['Team_b_PF'] = df.apply(lambda x: x.LPF if x.id_diff<0 else x.WPF, axis =1)
    
    df['Score_diff'] = df.apply(lambda x: x.WScore - x.LScore if x.id_diff <0 else x.LScore-x.WScore, axis = 1) 
    for i in ['Season','WTeamID','LTeamID']:
        df[i] = df[i].apply(lambda x: str(x))
    df['ID'] = df.apply(lambda x: x.Season+'_'+x.WTeamID+'_'+x.LTeamID if x.id_diff <0 else x.Season +'_'+ x.LTeamID+'_'+x.WTeamID,axis= 1)
    
    return df

In [13]:
#Apply score_diff to both regular season and tourney
df_list = [df_regular_results,df_tourney_results]

df_sdiff = pd.DataFrame()
for i in df_list:
    temp = score_diff(i)
    print(temp.shape)
    df_sdiff = df_sdiff.append(temp)

(87504, 62)
(1115, 62)


## Thoughts about "duplicates"

## 1. A 对 B 对阵的次数 - New Label variable (maybe used in some way)
## 2. ID 保留，创建新的variable, ID_DayNum (unique), used to merge

In [14]:
#Keep only ID and Score_diff and merged with DataMart
df_sdiff_formerge = df_sdiff[[
                                'ID', 
                                'DayNum',
                                'Team_a_Loc',
                                'Team_a_FGM2_rate',
                                'Team_a_FGM3_rate',
                                'Team_a_FTM_count',
                                'Team_a_OR',
                                'Team_a_DR',
                                'Team_a_Ast',
                                'Team_a_TO',
                                'Team_a_Stl',
                                'Team_a_Blk',
                                'Team_a_PF',
                                'Team_b_Loc',
                                'Team_b_FGM2_rate',
                                'Team_b_FGM3_rate',
                                'Team_b_FTM_count',
                                'Team_b_OR',
                                'Team_b_DR',
                                'Team_b_Ast',
                                'Team_b_TO',
                                'Team_b_Stl',
                                'Team_b_Blk',
                                'Team_b_PF',
                                'Score_diff',
                                'Label',
                                'win'
                             ]]

df_datamart_add1 = pd.merge(left = df_datamart,right = df_sdiff_formerge, on = 'ID',how = 'left')

print(df_datamart_add1.shape)

print(df_datamart_add1.head())

(2376273, 30)
               ID  Season  TeamID_a  TeamID_b  DayNum Team_a_Loc  \
0  1985_1101_1102    1985      1101      1102     NaN        NaN   
1  1985_1101_1103    1985      1101      1103     NaN        NaN   
2  1985_1101_1104    1985      1101      1104     NaN        NaN   
3  1985_1101_1105    1985      1101      1105     NaN        NaN   
4  1985_1101_1106    1985      1101      1106     NaN        NaN   

   Team_a_FGM2_rate  Team_a_FGM3_rate  Team_a_FTM_count  Team_a_OR  ...  \
0               NaN               NaN               NaN        NaN  ...   
1               NaN               NaN               NaN        NaN  ...   
2               NaN               NaN               NaN        NaN  ...   
3               NaN               NaN               NaN        NaN  ...   
4               NaN               NaN               NaN        NaN  ...   

   Team_b_OR  Team_b_DR  Team_b_Ast  Team_b_TO  Team_b_Stl  Team_b_Blk  \
0        NaN        NaN         NaN        NaN      

In [15]:
check_missing(df_datamart_add1)

,col_names,col_dtypes,col_na_count,col_unique,count,mean,std,min,25%,50%,75%,max
ID,ID,object,0,2350635,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Season,Season,int64,0,35,2376273.0,2002.099245,10.102577,1985.000000,1993.000000,2002.000000,2011.000000,2019.000000
TeamID_a,TeamID_a,int64,0,366,2376273.0,1222.687155,86.387214,1101.000000,1150.000000,1208.000000,1284.000000,1466.000000
TeamID_b,TeamID_b,int64,0,366,2376273.0,1345.322526,86.382706,1102.000000,1284.000000,1360.000000,1418.000000,1467.000000
DayNum,DayNum,float64,2287654,145,88619.0,71.969747,36.045945,0.000000,40.000000,75.000000,103.000000,154.000000
Team_a_Loc,Team_a_Loc,object,2287654,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Team_a_FGM2_rate,Team_a_FGM2_rate,float64,2287654,740,88619.0,0.484551,0.094958,0.060606,0.420000,0.483871,0.547619,0.900000
Team_a_FGM3_rate,Team_a_FGM3_rate,float64,2287654,361,88619.0,0.344142,0.117848,0.000000,0.266667,0.333333,0.421053,1.000000
Team_a_FTM_count,Team_a_FTM_count,float64,2287654,64,88619.0,20.204595,7.985495,0.000000,14.000000,20.000000,25.000000,67.000000
Team_a_OR,Team_a_OR,float64,2287654,37,88619.0,10.946908,4.152362,0.000000,8.000000,11.000000,14.000000,38.000000


In [16]:
same_id_game_count = pd.DataFrame(data = df_datamart_add1['ID'].value_counts()).rename(columns={'ID':'same_id_game_count'})
same_id_game_count['ID'] = same_id_game_count.index

In [17]:
df_datamart_add2 = pd.merge(left = df_datamart_add1,right = same_id_game_count, on = 'ID',how = 'left')
df_datamart_add2['DayNum'].fillna(999,inplace=True)
df_datamart_add2['DayNum'] = df_datamart_add2['DayNum'].astype(np.int64)
check_missing(df_datamart_add2)

,col_names,col_dtypes,col_na_count,col_unique,count,mean,std,min,25%,50%,75%,max
ID,ID,object,0,2350635,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Season,Season,int64,0,35,2376273.0,2002.099245,10.102577,1985.000000,1993.000000,2002.000000,2011.000000,2019.000000
TeamID_a,TeamID_a,int64,0,366,2376273.0,1222.687155,86.387214,1101.000000,1150.000000,1208.000000,1284.000000,1466.000000
TeamID_b,TeamID_b,int64,0,366,2376273.0,1345.322526,86.382706,1102.000000,1284.000000,1360.000000,1418.000000,1467.000000
DayNum,DayNum,int64,0,146,2376273.0,964.428007,175.791151,0.000000,999.000000,999.000000,999.000000,999.000000
Team_a_Loc,Team_a_Loc,object,2287654,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Team_a_FGM2_rate,Team_a_FGM2_rate,float64,2287654,740,88619.0,0.484551,0.094958,0.060606,0.420000,0.483871,0.547619,0.900000
Team_a_FGM3_rate,Team_a_FGM3_rate,float64,2287654,361,88619.0,0.344142,0.117848,0.000000,0.266667,0.333333,0.421053,1.000000
Team_a_FTM_count,Team_a_FTM_count,float64,2287654,64,88619.0,20.204595,7.985495,0.000000,14.000000,20.000000,25.000000,67.000000
Team_a_OR,Team_a_OR,float64,2287654,37,88619.0,10.946908,4.152362,0.000000,8.000000,11.000000,14.000000,38.000000


In [18]:
df_datamart_add2['ID_DayNum'] = df_datamart_add2.apply(lambda x: x.ID + '_' + str(x.DayNum), axis=1)
df_datamart_add2

,ID,Season,TeamID_a,TeamID_b,DayNum,Team_a_Loc,Team_a_FGM2_rate,Team_a_FGM3_rate,Team_a_FTM_count,Team_a_OR,...,Team_b_Ast,Team_b_TO,Team_b_Stl,Team_b_Blk,Team_b_PF,Score_diff,Label,win,same_id_game_count,ID_DayNum
0,1985_1101_1102,1985,1101,1102,999,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1985_1101_1102_999
1,1985_1101_1103,1985,1101,1103,999,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1985_1101_1103_999
2,1985_1101_1104,1985,1101,1104,999,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1985_1101_1104_999
3,1985_1101_1105,1985,1101,1105,999,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1985_1101_1105_999
4,1985_1101_1106,1985,1101,1106,999,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1985_1101_1106_999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2376268,2019_1464_1466,2019,1464,1466,999,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2019_1464_1466_999
2376269,2019_1464_1467,2019,1464,1467,999,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2019_1464_1467_999
2376270,2019_1465_1466,2019,1465,1466,999,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2019_1465_1466_999
2376271,2019_1465_1467,2019,1465,1467,999,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2019_1465_1467_999


### [MNCAATourneySeed.csv]文件，战绩统计，
### 1. 出现a,b的为末位排名，进行次数统计
### 2. 出现a,b后胜利次数统计
### 3. 锦标赛排名分值：各个地区（四个w,x,y,z）中的排名有参考价值，故应队排名加次数赋值，合计（例：w01-16scores），未能进入到锦标赛的队伍，此项分值为0

In [19]:
df_tourneyseed = pd.read_csv(input_path_mdata + 'MNCAATourneySeeds.csv', engine = 'python')
df_tourneyseed

,Season,Seed,TeamID
0,1985,W01,1207
1,1985,W02,1210
2,1985,W03,1228
3,1985,W04,1260
4,1985,W05,1374
...,...,...,...
2281,2019,Z12,1332
2282,2019,Z13,1414
2283,2019,Z14,1330
2284,2019,Z15,1159


In [20]:
# 取出a,b 队伍
df_tourneyseed['Seed_first_four'] = df_tourneyseed.apply(lambda x: 1 if 'a' in x.Seed else( 1 if 'b' in x.Seed else 0) , axis=1)

In [21]:
# 定义季后赛分数
df_tourneyseed['Seed_Num'] = df_tourneyseed['Seed'].apply(lambda x: int(re.findall(r'[0-9]+',x)[0]))
df_tourneyseed['Seed_score'] = df_tourneyseed['Seed_Num'].apply(lambda x: 17-x )

In [22]:
df_tourneyseed.head(5)

,Season,Seed,TeamID,Seed_first_four,Seed_Num,Seed_score
0,1985,W01,1207,0,1,16
1,1985,W02,1210,0,2,15
2,1985,W03,1228,0,3,14
3,1985,W04,1260,0,4,13
4,1985,W05,1374,0,5,12


In [23]:
#Set Season offset in order to calculate the most recent 5 year's seed score
df_tourneyseed['offset'] = df_tourneyseed['Season'] -4
combined = pd.merge(left = df_tourneyseed, right = df_tourneyseed, on = 'TeamID', how = 'left')
combined.head()

,Season_x,Seed_x,TeamID,Seed_first_four_x,Seed_Num_x,Seed_score_x,offset_x,Season_y,Seed_y,Seed_first_four_y,Seed_Num_y,Seed_score_y,offset_y
0,1985,W01,1207,0,1,16,1981,1985,W01,0,1,16,1981
1,1985,W01,1207,0,1,16,1981,1986,X04,0,4,13,1982
2,1985,W01,1207,0,1,16,1981,1987,X01,0,1,16,1983
3,1985,W01,1207,0,1,16,1981,1988,W08,0,8,9,1984
4,1985,W01,1207,0,1,16,1981,1989,W01,0,1,16,1985


In [24]:
#Calculated total average seed score: for each team in each season, the average seed score in the previous year
combined_totalavg = combined[combined.Season_y <= combined.Season_x]
total_avg = combined_totalavg.groupby(['TeamID','Season_x'])['Seed_score_y'].mean().reset_index().rename(columns = {'Seed_score_y':'Seed_total_count_avg'})
total_avg.head()

,TeamID,Season_x,Seed_total_count_avg
0,1101,2019,2.0
1,1102,2004,6.0
2,1102,2006,5.0
3,1103,1986,2.0
4,1103,2009,3.0


In [25]:
combined_fiveavg = combined[(combined.Season_y >= combined.offset_x)&(combined.Season_y <= combined.Season_x)]
five_year_avg = combined_fiveavg.groupby(['TeamID','Season_x'])['Seed_score_y'].sum().reset_index().rename(columns = {'Seed_score_y':'Seed_five_year_sum'})
five_year_avg['Seed_five_year_avg'] = five_year_avg['Seed_five_year_sum']/5
five_year_avg.head()

,TeamID,Season_x,Seed_five_year_sum,Seed_five_year_avg
0,1101,2019,2,0.4
1,1102,2004,6,1.2
2,1102,2006,10,2.0
3,1103,1986,2,0.4
4,1103,2009,4,0.8


In [26]:
five_count_avg = combined_fiveavg.groupby(['TeamID','Season_x'])['Seed_score_y'].mean().reset_index().rename(columns = {'Seed_score_y':'Seed_five_count_avg'})
five_count_avg.head()

,TeamID,Season_x,Seed_five_count_avg
0,1101,2019,2.0
1,1102,2004,6.0
2,1102,2006,5.0
3,1103,1986,2.0
4,1103,2009,4.0


In [27]:
five_avg = pd.merge(left = five_year_avg, right = five_count_avg, on = ['TeamID','Season_x'] ,how = 'left')
five_avg.head(20)

,TeamID,Season_x,Seed_five_year_sum,Seed_five_year_avg,Seed_five_count_avg
0,1101,2019,2,0.4,2.000000
1,1102,2004,6,1.2,6.000000
2,1102,2006,10,2.0,5.000000
3,1103,1986,2,0.4,2.000000
4,1103,2009,4,0.8,4.000000
5,1103,2011,6,1.2,3.000000
6,1103,2013,11,2.2,3.666667
7,1104,1985,10,2.0,10.000000
8,1104,1986,22,4.4,11.000000
9,1104,1987,37,7.4,12.333333


In [28]:
seed_avg = pd.merge(left= five_avg, right = total_avg, on = ['TeamID', 'Season_x'], how='left').rename(columns={'Season_x': 'Season'})
seed_avg.head()

,TeamID,Season,Seed_five_year_sum,Seed_five_year_avg,Seed_five_count_avg,Seed_total_count_avg
0,1101,2019,2,0.4,2.0,2.0
1,1102,2004,6,1.2,6.0,6.0
2,1102,2006,10,2.0,5.0,5.0
3,1103,1986,2,0.4,2.0,2.0
4,1103,2009,4,0.8,4.0,3.0


In [29]:
df_tourneyseed = pd.merge(left=df_tourneyseed,right=seed_avg, on=['TeamID', 'Season'], how='left')
df_tourneyseed

,Season,Seed,TeamID,Seed_first_four,Seed_Num,Seed_score,offset,Seed_five_year_sum,Seed_five_year_avg,Seed_five_count_avg,Seed_total_count_avg
0,1985,W01,1207,0,1,16,1981,16,3.2,16.0,16.000000
1,1985,W02,1210,0,2,15,1981,15,3.0,15.0,15.000000
2,1985,W03,1228,0,3,14,1981,14,2.8,14.0,14.000000
3,1985,W04,1260,0,4,13,1981,13,2.6,13.0,13.000000
4,1985,W05,1374,0,5,12,1981,12,2.4,12.0,12.000000
...,...,...,...,...,...,...,...,...,...,...,...
2281,2019,Z12,1332,0,12,5,2015,44,8.8,11.0,10.500000
2282,2019,Z13,1414,0,13,4,2015,8,1.6,4.0,4.000000
2283,2019,Z14,1330,0,14,3,2015,3,0.6,3.0,4.900000
2284,2019,Z15,1159,0,15,2,2015,2,0.4,2.0,1.333333


In [32]:
df_datamart_add3 =pd.merge(left=df_datamart_add2, right=df_tourneyseed, left_on=['TeamID_a', 'Season'],right_on = ['TeamID', 'Season'], how='left').rename(columns={'Seed':'Seed_a', 
                                                                                                                             'Seed_first_four':'Seed_first_four_a',
                                                                                                                             'Seed_Num':'Seed_Num_a',
                                                                                                                             'Seed_score':'Seed_score_a',
                                                                                                                             'offset':'offset_a',
                                                                                                                             'Seed_five_year_sum':'Seed_five_year_sum_a',
                                                                                                                             'Seed_five_year_avg':'Seed_five_year_avg_a',
                                                                                                                             'Seed_five_count_avg':'Seed_five_count_avg_a',
                                                                                                                             'Seed_total_count_avg':'Seed_total_count_avg_a'})
df_datamart_add3

,ID,Season,TeamID_a,TeamID_b,DayNum,Team_a_Loc,Team_a_FGM2_rate,Team_a_FGM3_rate,Team_a_FTM_count,Team_a_OR,...,Seed_a,TeamID,Seed_first_four_a,Seed_Num_a,Seed_score_a,offset_a,Seed_five_year_sum_a,Seed_five_year_avg_a,Seed_five_count_avg_a,Seed_total_count_avg_a
0,1985_1101_1102,1985,1101,1102,999,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1985_1101_1103,1985,1101,1103,999,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1985_1101_1104,1985,1101,1104,999,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1985_1101_1105,1985,1101,1105,999,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1985_1101_1106,1985,1101,1106,999,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2376268,2019_1464_1466,2019,1464,1466,999,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2376269,2019_1464_1467,2019,1464,1467,999,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2376270,2019_1465_1466,2019,1465,1466,999,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2376271,2019_1465_1467,2019,1465,1467,999,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
df_datamart_add4 =pd.merge(left=df_datamart_add3, right=df_tourneyseed, left_on=['TeamID_b', 'Season'],right_on = ['TeamID', 'Season'], how='left').rename(columns={'Seed':'Seed_b', 
                                                                                                                             'Seed_first_four':'Seed_first_four_b',
                                                                                                                             'Seed_Num':'Seed_Num_b',
                                                                                                                             'Seed_score':'Seed_score_b',
                                                                                                                             'offset':'offset_b',
                                                                                                                             'Seed_five_year_sum':'Seed_five_year_sum_b',
                                                                                                                             'Seed_five_year_avg':'Seed_five_year_avg_b',
                                                                                                                             'Seed_five_count_avg':'Seed_five_count_avg_b',
                                                                                                                             'Seed_total_count_avg':'Seed_total_count_avg_b'})
df_datamart_add4.drop(columns = ['TeamID_x','Seed_Num_a','offset_a', 'Seed_five_year_sum_a',
                                 'TeamID_y','Seed_Num_b','offset_b', 'Seed_five_year_sum_b'], inplace=True)

In [36]:
df_datamart_add4.shape

(2376273, 44)

In [38]:
check_missing(df_datamart_add4)

,col_names,col_dtypes,col_na_count,col_unique,count,mean,std,min,25%,50%,75%,max
ID,ID,object,0,2350635,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Season,Season,int64,0,35,2376273.0,2002.099245,10.102577,1985.000000,1993.000000,2002.000000,2011.000000,2019.000000
TeamID_a,TeamID_a,int64,0,366,2376273.0,1222.687155,86.387214,1101.000000,1150.000000,1208.000000,1284.000000,1466.000000
TeamID_b,TeamID_b,int64,0,366,2376273.0,1345.322526,86.382706,1102.000000,1284.000000,1360.000000,1418.000000,1467.000000
DayNum,DayNum,int64,0,146,2376273.0,964.428007,175.791151,0.000000,999.000000,999.000000,999.000000,999.000000
Team_a_Loc,Team_a_Loc,object,2287654,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Team_a_FGM2_rate,Team_a_FGM2_rate,float64,2287654,740,88619.0,0.484551,0.094958,0.060606,0.420000,0.483871,0.547619,0.900000
Team_a_FGM3_rate,Team_a_FGM3_rate,float64,2287654,361,88619.0,0.344142,0.117848,0.000000,0.266667,0.333333,0.421053,1.000000
Team_a_FTM_count,Team_a_FTM_count,float64,2287654,64,88619.0,20.204595,7.985495,0.000000,14.000000,20.000000,25.000000,67.000000
Team_a_OR,Team_a_OR,float64,2287654,37,88619.0,10.946908,4.152362,0.000000,8.000000,11.000000,14.000000,38.000000


In [39]:
df_datamart_add4.columns

Index(['ID', 'Season', 'TeamID_a', 'TeamID_b', 'DayNum', 'Team_a_Loc',
       'Team_a_FGM2_rate', 'Team_a_FGM3_rate', 'Team_a_FTM_count', 'Team_a_OR',
       'Team_a_DR', 'Team_a_Ast', 'Team_a_TO', 'Team_a_Stl', 'Team_a_Blk',
       'Team_a_PF', 'Team_b_Loc', 'Team_b_FGM2_rate', 'Team_b_FGM3_rate',
       'Team_b_FTM_count', 'Team_b_OR', 'Team_b_DR', 'Team_b_Ast', 'Team_b_TO',
       'Team_b_Stl', 'Team_b_Blk', 'Team_b_PF', 'Score_diff', 'Label', 'win',
       'same_id_game_count', 'ID_DayNum', 'Seed_a', 'Seed_first_four_a',
       'Seed_score_a', 'Seed_five_year_avg_a', 'Seed_five_count_avg_a',
       'Seed_total_count_avg_a', 'Seed_b', 'Seed_first_four_b', 'Seed_score_b',
       'Seed_five_year_avg_b', 'Seed_five_count_avg_b',
       'Seed_total_count_avg_b'],
      dtype='object')

In [40]:
df_datamart_v1 = df_datamart_add4[[ 
                                    'ID', 
                                    'ID_DayNum',
                                    'Season', 
                                    'Seed_a', 
                                    'Seed_first_four_a',
                                    'Seed_score_a', 
                                    'Seed_five_year_avg_a', 
                                    'Seed_five_count_avg_a',
                                    'Seed_total_count_avg_a', 
                                    'Seed_b', 
                                    'Seed_first_four_b', 
                                    'Seed_score_b',
                                    'Seed_five_year_avg_b', 
                                    'Seed_five_count_avg_b',
                                    'Seed_total_count_avg_b',
                                    'TeamID_a', 
                                    'TeamID_b', 
                                    'DayNum', 
                                    'Team_a_Loc',
                                    'Team_a_FGM2_rate', 
                                    'Team_a_FGM3_rate', 
                                    'Team_a_FTM_count', 
                                    'Team_a_OR',
                                    'Team_a_DR', 
                                    'Team_a_Ast', 
                                    'Team_a_TO', 
                                    'Team_a_Stl', 
                                    'Team_a_Blk',
                                    'Team_a_PF', 
                                    'Team_b_Loc', 
                                    'Team_b_FGM2_rate', 
                                    'Team_b_FGM3_rate',
                                    'Team_b_FTM_count', 
                                    'Team_b_OR', 
                                    'Team_b_DR', 
                                    'Team_b_Ast', 
                                    'Team_b_TO',
                                    'Team_b_Stl', 
                                    'Team_b_Blk', 
                                    'Team_b_PF', 
                                    'Score_diff', 
                                    'Label', 
                                    'win'
                                  ]]

In [41]:
check_missing(df_datamart_v1)

,col_names,col_dtypes,col_na_count,col_unique,count,mean,std,min,25%,50%,75%,max
ID,ID,object,0,2350635,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ID_DayNum,ID_DayNum,object,0,2376273,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Season,Season,int64,0,35,2376273.0,2002.099245,10.102577,1985.000000,1993.000000,2002.000000,2011.000000,2019.000000
Seed_a,Seed_a,object,1970302,90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Seed_first_four_a,Seed_first_four_a,float64,1970302,2,405971.0,0.041387,0.199184,0.000000,0.000000,0.000000,0.000000,1.000000
Seed_score_a,Seed_score_a,float64,1970302,16,405971.0,8.384330,4.736752,1.000000,4.000000,8.000000,13.000000,16.000000
Seed_five_year_avg_a,Seed_five_year_avg_a,float64,1970302,80,405971.0,5.366471,4.592755,0.200000,1.200000,4.000000,8.800000,16.000000
Seed_five_count_avg_a,Seed_five_count_avg_a,float64,1970302,124,405971.0,8.326685,4.424765,1.000000,4.000000,9.000000,12.000000,16.000000
Seed_total_count_avg_a,Seed_total_count_avg_a,float64,1970302,670,405971.0,8.287453,4.240204,1.000000,4.000000,9.214286,12.000000,16.000000
Seed_b,Seed_b,object,1935103,90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
df_datamart_v1.to_csv(output_path + 'datamart_v1.csv')